<a href="https://colab.research.google.com/github/generalvro/INVEX/blob/main/ME2M_ODB_A_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Subir archivo manualmente
from google.colab import files
import pandas as pd
from datetime import datetime
import pytz

uploaded = files.upload()  # Aquí seleccionas tu .xlsx
nombre_archivo = list(uploaded.keys())[0]

# 2. Leer Excel como texto
df = pd.read_excel(nombre_archivo, dtype=str)
df = df.astype(str)  # asegurar que todas las columnas son texto

# 3. Ajuste sencillo y robusto para "Cantidad de pedido" y "Por entregar (cantidad)"
for col in ["Cantidad de pedido", "Por entregar (cantidad)"]:
    if col in df.columns:
        s = df[col].fillna('').astype(str)

        # 3.1 si comienza con coma (",432") => "0,432"
        s = s.str.replace(r'^\s*,', '0,', regex=True)

        # 3.2 Si es un número sin separadores que empieza por 0 y tiene hasta 4 dígitos (ej "0432")
        #      lo interpretamos como 0.xxx -> "0.432"
        s = s.str.replace(r'^0([0-9]{1,4})$', r'0.\1', regex=True)

        # 3.3 Si la celda contiene coma (formato SAP: miles con '.' y decimal con ','),
        #      quitar todos los puntos (miles) y luego cambiar coma por punto.
        mask_comma = s.str.contains(',', regex=False)
        s_comma = s[mask_comma].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
        s.loc[mask_comma] = s_comma

        # 3.4 Si no tenía coma y tenía puntos (casos como "0.432" o "432" o "1.234"),
        #      dejamos tal cual (no tocamos más para no romper decimales ya correctos).
        df[col] = s

# 4. Normalizar fechas a formato PostgreSQL (yyyy-mm-dd)
for col in ["Fecha documento", "Fecha de entrega"]:
    if col in df.columns:
        df[col] = pd.to_datetime(
            df[col], errors="coerce", dayfirst=True
        ).dt.strftime("%Y-%m-%d")

# 5. Crear columna "CODIGO_PROVEEDOR" con los primeros 7 caracteres de la primera columna (columna A)
col_a = df.columns[0]
df["CODIGO_PROVEEDOR"] = df[col_a].str[:7]

# 6. Exportar a CSV con fecha de la máquina (zona horaria Bogotá)
tz = pytz.timezone("America/Bogota")
fecha_hoy = datetime.now(tz).strftime("%d_%m_%Y")
nombre_salida = f"ME2M_ODB_{fecha_hoy}.csv"

df.to_csv(nombre_salida, sep=";", index=False, encoding="utf-8")

# 7. Descargar el archivo
files.download(nombre_salida)
